In [1]:
import gc
import os
import sys
import platform
import logging
import argparse
from datetime import datetime, date, timedelta
import pendulum
import pandas as pd
import numpy as np

repo_path = '/home/notebook'
sys.path.append(os.path.join(repo_path, 'test'))


# from common.cpoall_aggregator import run_daily_tms_analysis
# Import necessary files and its respective functions
from common.db_operations import connect_to_trino,execute_query, fetch_distinct_ids_for_day_trino,fetch_data_for_day_trino, write_df_to_iceberg, parse_arguments, get_target_date
from common.cpoall_aggregator import run_daily_tms_analysis,CORE_COLS

In [ ]:
# # List of table names to be dropped
# tables_to_drop = [
#     # "ac_bcs_temperature_delta",
#     # "bcs_pack_temperature_analysis",
#     # "braking_report",
#     # "braking_analysis_results",
#     # "charging_graph",
#     # "charging_session_stats",
#     # "charging_sessions",
#     "energy_mileage_hourly",
#     # "energy_mileage_quarterly",
#     # "energy_stats_report",
#     # "soc_units_report",
#     # "soc_vs_odometer"
# ]

# conn = connect_to_trino()

# for table in tables_to_drop:
#     # Using an f-string to inject the table name into the schema path
#     sql = f"DROP TABLE IF EXISTS adhoc.facts_prod.{table}"
    
#     print(f"Dropping table: {table}...") # Optional: for logging
#     execute_query(conn, sql, return_results=False)

# conn.close()
# print("Cleanup complete.")

2025-12-23 12:02:49 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-23 12:02:49 - INFO - ✅ STEP 1: Connected to Trino
2025-12-23 12:02:49 - INFO - ⚙️ Executing query...
2025-12-23 12:02:49 - INFO - ✅ Query executed successfully (no results expected)!


Dropping table: energy_mileage_hourly...
Cleanup complete.


In [3]:
wait

NameError: name 'wait' is not defined

In [ ]:
CORE_COLS = [
    # Metadata & Base Info
    "id", "timestamp", "dt",
    "sequence","number_of_can_ids","number_of_records",
    
    # cell voltage/temperature
    "air_compreesor_temperature", "motor_temperature", "igbt_temperature",
    "gunc_dcp_temperature", "gunc_dcm_temperature", "gunb_dcm_temperature",
    "guna_dcp_temperature", "guna_dcm_temperature", "gunb_dcp_temperature",
    "pack1_cell_min_temperature", "pack1_cellmax_temperature",
    "pack1_cell_min_voltage", "pack1_cell_max_voltage",
    "pack_cellvoltage_1","pack_temperature1",
    "cell_min_voltage", "cell_max_voltage",
    "pack1_celltemperature_cellnumber", "pack1_maxtemperature_cell_number",
    "pack1_cellminvoltagecellnumber", "pack1_cellmax_voltagecellnumber",
    "cellminvoltagecellnumber", "cellmax_voltagecellnumber",

    # Binary/Categorical_status
    "gun_connection_status", "vehiclereadycondition", "ignitionstatus",
    "pack_negative_contactors_status", "vcuself_teststatus", "mcuself_teststatus",
    "parking_brake_status", "chargingcontactor1negative", "chargingcontactor2positive",
    "charging_contactor_3_negative", "charging_contactor_3_positive",
    "chargingcontactor2negative", "chargingcontactor1positive",
    "batterytotalnegativecontactor", "dcdc_statusandfailure", "kneeling_request",
    "systeminterlockstate", "autoholdunenable", "oilpumpcondition",
    "airpumpcondition", "insulation_status",

    # Commands
    "air_compressor_enable_command", "steering_pump_enable_command", "dcdc_enable_command",

    # Electrical
    "output_phase_currant", "output_phase_voltage", "dcdcbusvoltage",
    "dcdc_out_put_currant", "dcdc_out_put_volatge", "maxavailableshorttermcharging",
    "maxavailableshorttimedischarge", "soh", "bat_voltage", "bat_soc",
    "total_battery_current", "insulation__value",

    # DriveMode
    "odometerreading", "brakepedalpos", "gear_position", "vehicle_operation_mode",
    "motor_torque", "motor_rpm", "vehicle_speed_vcu", "distancetoempty",
    "brake_pedal", "accelarationpedal", "front_air_pressure", "rear_air_pressure",
    "steering_pump_rpm",

    # Life Signals (from previous list)
    "airpumpdcaclifesignal", "lowvoltagedcaclifesignal", "ebsredwarningsignal",
    "abs_ebsamberwarningsignal", "bmslifesignal", "dcdclifesignal", "lifesignalofinsulation",

    # Alarms & Faults (from previous list)
    "polehightemperaturealarm", "batteryhightemperaturealarm", "steeringcontroller_failurealarm",
    "battery_function_alarm", "powertrain_failure_alarm", "temperaturedifferencealarm",
    "singlevoltagedifferencealarm", "chargingcurrentalarm", "batterypackundervoltagealarm",
    "batterypackovervoltagealarm", "monomerovervoltagealarm", "monomerundervoltagealarm",
    "dischargecurrentalarm", "soclowalarm", "sochighalarm", "brake_pad_worn_out_alarm",
    "brakingsystemmallfunction_alarm", "side_door_panel_open_alarm", "hightempalarmofcharginggun",
    "lowpressureoilpumpfaultcode", "bms_fault_code", "vcu_fault_code"
]

In [ ]:
conn = connect_to_trino()   # your DB function
day_str = "2025-10-29"
core_cols = CORE_COLS
source_schema = "facts_prod"
source_table = "can_parsed_output_all"
ids = ['6']

df_raw1 = fetch_data_for_day_trino(conn=conn,day_str=day_str,ids=ids,core_cols=core_cols,table=source_table,schema=source_schema,)

2025-12-22 20:19:59 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-22 20:19:59 - INFO - ✅ STEP 1: Connected to Trino


In [ ]:
df_chk = df_raw1.isnull().sum()*100.0/len(df_raw1)
df_chk[df_chk>50.0].sort_values(ascending=False)

pack1_cell_min_voltage             100.000000
pack1_cell_max_voltage             100.000000
pack1_cellminvoltagecellnumber     100.000000
pack1_cellmax_voltagecellnumber    100.000000
pack_cellvoltage_1                  98.977096
pack_temperature1                   91.327552
insulation_status                   72.181454
insulation__value                   72.181454
lifesignalofinsulation              72.181454
dcdc_statusandfailure               50.744941
dcdcbusvoltage                      50.744941
dcdc_out_put_currant                50.744941
dcdc_out_put_volatge                50.744941
dcdclifesignal                      50.744941
dtype: float64

In [ ]:
# display(df_raw.lifesignalofinsulation.unique())
display(df_raw1.number_of_records.unique())

array([ 13,  17,  19,  15,  18,  20,  73,  31,  56,  74,  57,  14,  69,
        67,  65,  58,  86,  23,  52,  30,  53,  83,  85,  64,  43,  79,
        61,  50,  55,  60,  54,  51,  27,  26,  89,  25,  46,  72,  82,
        49,  29,  41,  37,  38,  48,  70,  63,  39,   6,  11,  59,  68,
        75,  78,  84,  95,  93,  47,  81,  66,  33,  44,  80,  40,  35,
        76,  92,  62,  88,  16,   3,   9, 105,  45,  24,  90,  12,  77,
        94,  36,  42,  71,  32,  28,  21,  34,  22,  91,  87, 108,  99,
       107,  97, 100,   1,  96, 102, 101,  98, 110, 106,   5,   7,   8,
        10,   2, 103, 112, 117,   4])

In [ ]:
conn = connect_to_trino()   # your DB function
day_str = "2025-11-29"
core_cols = CORE_COLS
source_schema = "facts_prod"
source_table = "can_parsed_output_all"
ids = ['6','18','19']

df_raw2 = fetch_data_for_day_trino(conn=conn,day_str=day_str,ids=ids,core_cols=core_cols,table=source_table,schema=source_schema,)

2025-12-22 20:20:04 - INFO - 🔌 STEP 1: Connecting to Trino...
2025-12-22 20:20:04 - INFO - ✅ STEP 1: Connected to Trino


In [ ]:
df_chk = df_raw2.isnull().sum()*100.0/len(df_raw2)
df_chk[df_chk>50.0].sort_values(ascending=False)

cell_min_voltage             100.000000
cell_max_voltage             100.000000
cellminvoltagecellnumber     100.000000
cellmax_voltagecellnumber    100.000000
pack_cellvoltage_1            99.216730
pack_temperature1             92.599101
insulation_status             78.075130
insulation__value             78.075130
lifesignalofinsulation        78.075130
dcdc_statusandfailure         58.527686
dcdcbusvoltage                58.527686
dcdc_out_put_currant          58.527686
dcdc_out_put_volatge          58.527686
dcdclifesignal                58.527686
dtype: float64

In [ ]:
display(df_raw2.total_battery_current.unique())

array([  nan, 124.4, 304. , ..., 399.8, 384. , 374.7], shape=(6685,))

In [ ]:
import re
import numpy as np
import pandas as pd

SPEC_PATH = '/home/notebook/test/common/canId_conversions.txt'
PCTS = [0.01, 0.05, 0.50, 0.95, 0.99]

def parse_expected_ranges(path: str) -> dict:
    """
    Parses lines like:
      SG_ Bat_Voltage :[0|1000]
    Returns: dict {normalized_name: (min,max)}
    """
    pat = re.compile(r"^\s*SG_\s+([A-Za-z0-9_]+)\s*:\s*\[([\-0-9.]+)\|([\-0-9.]+)\]\s*$")
    out = {}
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            m = pat.match(line)
            if not m:
                continue
            name, lo, hi = m.group(1), float(m.group(2)), float(m.group(3))
            key = re.sub(r"__+", "_", name.strip().lower())
            out[key] = (lo, hi)
    return out

EXPECTED = parse_expected_ranges(SPEC_PATH)

# Optional: add pattern fallback for families of columns if needed
PATTERN_EXPECTED = [
    (re.compile(r"^pack_temperature\d+$"), (-40.0, 215.0)),
    (re.compile(r"^pack_cellvoltage_\d+$"), (0.0, 600.0)),
]

def expected_range_for_column(col: str) -> tuple[float, float] | None:
    c = re.sub(r"__+", "_", col.lower())
    if c in EXPECTED:
        return EXPECTED[c]
    for pat, rng in PATTERN_EXPECTED:
        if pat.match(c):
            return rng
    return None

def profile_with_dbc_bounds(df: pd.DataFrame, cols: list[str]) -> pd.DataFrame:
    rows = []
    n = len(df)

    for col in cols:
        s_num = pd.to_numeric(df[col], errors="coerce")
        nn = s_num.notna().sum()

        exp = expected_range_for_column(col)
        dbc_min, dbc_max = (exp if exp else (np.nan, np.nan))

        if nn == 0:
            rows.append({
                "column": col,
                "non_null_pct": 0.0,
                "dbc_min": dbc_min,
                "obs_min": np.nan,                
                "p01": np.nan, "p05": np.nan, "p50": np.nan, "p95": np.nan, "p99": np.nan,
                "obs_max": np.nan,
                "dbc_max": dbc_max,
            })
            continue

        qs = s_num.quantile(PCTS).to_dict()
        rows.append({
            "column": col,
            "non_null_pct": (nn / n * 100) if n else np.nan,
            "dbc_min": dbc_min,            
            "obs_min": float(s_num.min()),
            "p01": float(qs[0.01]),
            "p05": float(qs[0.05]),
            "p50": float(qs[0.50]),
            "p95": float(qs[0.95]),
            "p99": float(qs[0.99]),
            "obs_max": float(s_num.max()),
            "dbc_max": dbc_max,            
        })

    out = pd.DataFrame(rows)

    # Nice ordering: low coverage first, then widest observed spread
    spread = (out["obs_max"] - out["obs_min"]).replace([np.inf, -np.inf], np.nan)
    out["obs_spread"] = spread
    out = out.sort_values(["non_null_pct", "obs_spread"], ascending=[True, False]).drop(columns=["obs_spread"])
    return out.reset_index(drop=True)

# ---- Usage ----
ignore = {"cell_min_voltage","cell_max_voltage","cellminvoltagecellnumber","cellmax_voltagecellnumber"}
cols = [c for c in df_raw2.columns if c not in ignore and c not in {"id","timestamp","dt","insert_timestamp"}]

bounds_df = profile_with_dbc_bounds(df_raw2, cols)
# bounds_df.to_csv("quantiles_with_dbc_bounds.csv", index=False)
desired_order = [
    "column", "non_null_pct",
    "dbc_min", "obs_min",
    "p01", "p05", "p50", "p95", "p99",
    "obs_max", "dbc_max"
]
bounds_df = bounds_df[desired_order]
bounds_df.sort_values(by='dbc_max',ascending=True)

,column,non_null_pct,dbc_min,obs_min,p01,p05,p50,p95,p99,obs_max,dbc_max
15,maxavailableshorttermcharging,71.114403,-1000.0,-445.000,-445.00000,-445.000,-445.000000,0.00000,0.000000,0.00000,0.0
79,batteryhightemperaturealarm,86.999891,0.0,0.000,0.00000,0.000,0.000000,0.00000,0.000000,0.00000,0.0
10,odometerreading,52.187884,0.0,28957.625,28987.87500,29013.750,29719.875000,79376.50000,79409.625000,79427.00000,0.0
77,gun_connection_status,86.999891,0.0,0.000,0.00000,0.000,0.000000,1.00000,1.000000,1.00000,1.0
31,charging_contactor_3_positive,71.239927,0.0,0.000,0.00000,0.000,0.000000,1.00000,1.000000,1.00000,1.0
19,chargingcontactor2positive,71.114403,0.0,0.000,0.00000,0.000,0.000000,1.00000,1.000000,1.00000,1.0
20,chargingcontactor2negative,71.114403,0.0,0.000,0.00000,0.000,0.000000,1.00000,1.000000,1.00000,1.0
21,chargingcontactor1positive,71.114403,0.0,0.000,0.00000,0.000,0.000000,1.00000,1.000000,1.00000,1.0
22,batterytotalnegativecontactor,71.114403,0.0,0.000,1.00000,1.000,1.000000,1.00000,1.000000,1.00000,1.0
55,ignitionstatus,81.358840,0.0,0.000,0.00000,0.000,1.000000,1.00000,1.000000,1.00000,1.0


In [ ]:
DTYPE_RULES = {
    # --- Distances (store meters as integers) ---
    "odometerreading": {
        "variable_category": "continuous",
        "dtype": "UInt32",        # meters fits (up to ~4.29e9 m)
        "scale": 1000,            # km -> m
        "round": 0,
        "note": "Store as meters (km*1000) in UInt32",
    },
    "distancetoempty": {
        "variable_category": "continuous",
        "dtype": "UInt32",
        "scale": 1000,
        "round": 0,
        "note": "Store as meters (km*1000) in UInt32",
    },

    # --- Vehicle speed (drop decimals; store km/h as uint8) ---
    "vehicle_speed_vcu": {
        "variable_category": "continuous",
        "dtype": "UInt8",
        "scale": 1,
        "round": 0,
        "note": "Drop decimals; store integer speed (0-250)",
    },

    # --- Discrete / categorical enums (uint8) ---
    "gear_position": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},
    "kneeling_request": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    "ebsredwarningsignal": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "abs_ebsamberwarningsignal": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    "oilpumpcondition": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "airpumpcondition": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    "polehightemperaturealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "temperaturedifferencealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "singlevoltagedifferencealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "chargingcurrentalarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "dischargecurrentalarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    "vehiclereadycondition": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "vcuself_teststatus": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "mcuself_teststatus": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "systeminterlockstate": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "vehicle_operation_mode": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    "side_door_panel_open_alarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "brakingsystemmallfunction_alarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "brake_pad_worn_out_alarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    # Discovery-phase 0..255 domains (uint8)
    "soclowalarm": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0, "note": "0..255 domain"},
    "sochighalarm": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0, "note": "0..255 domain"},
    "bms_fault_code": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0, "note": "0..255 domain"},
    "vcu_fault_code": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0, "note": "0..255 domain"},
    "steeringcontroller_failurealarm": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0, "note": "0..255 domain"},
    "lowpressureoilpumpfaultcode": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0, "note": "0..255 domain"},

    # --- Binary signals (boolean) ---
    "pack_negative_contactors_status": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "chargingcontactor1negative": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "chargingcontactor1positive": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "chargingcontactor2positive": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "chargingcontactor2negative": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "batterytotalnegativecontactor": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "charging_contactor_3_negative": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "charging_contactor_3_positive": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "gun_connection_status": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "ignitionstatus": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "parking_brake_status": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "air_compressor_enable_command": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "steering_pump_enable_command": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "dcdc_enable_command": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "autoholdunenable": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},
    "powertrain_failure_alarm": {"variable_category": "binary", "dtype": "boolean", "scale": 1, "round": 0},

    # --- Continuous floats (Float32) ---
    "bat_soc": {"variable_category": "continuous", "dtype": "Float32", "scale": 1, "round": 1},
    "soh": {"variable_category": "continuous", "dtype": "Float32", "scale": 1, "round": 1},
    "bat_voltage": {"variable_category": "continuous", "dtype": "Float32", "scale": 1, "round": 1},
    "dcdc_out_put_volatge": {"variable_category": "continuous", "dtype": "Float32", "scale": 1, "round": 1},
    "dcdc_out_put_currant": {"variable_category": "continuous", "dtype": "Float32", "scale": 1, "round": 1},

    # Drop last decimal in current (as you noted)
    "total_battery_current": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0,
                              "note": "Drop decimals; store amps as int16 (engineering clip later)"},

    # --- Voltages in millivolts (UInt16) ---
    "pack1_cell_max_voltage": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1000, "round": 0,
                               "note": "Store V*1000 as mV"},
    "pack1_cell_min_voltage": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1000, "round": 0,
                               "note": "Store V*1000 as mV"},

    # --- More continuous (UInt16) ---
    "output_phase_currant": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "output_phase_voltage": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "dcdcbusvoltage": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "maxavailableshorttimedischarge": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "front_air_pressure": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "rear_air_pressure": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "motor_rpm": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "steering_pump_rpm": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0},
    "insulation__value": {"variable_category": "continuous", "dtype": "UInt16", "scale": 1, "round": 0,
                          "note": "Ohms-ish domain; confirm scaling if needed"},

    # --- Discrete indices (UInt16 / UInt8) ---
    "pack1_cellminvoltagecellnumber": {"variable_category": "discrete", "dtype": "UInt16", "scale": 1, "round": 0},
    "pack1_cellmax_voltagecellnumber": {"variable_category": "discrete", "dtype": "UInt16", "scale": 1, "round": 0},
    "pack1_maxtemperature_cell_number": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},
    "pack1_celltemperature_cellnumber": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},

    # --- Temperatures (store as Int16; keep 1°C resolution) ---
    "pack1_cellmax_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "pack1_cell_min_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "guna_dcp_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "gunb_dcp_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "gunc_dcp_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "guna_dcm_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "gunb_dcm_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "gunc_dcm_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "air_compreesor_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "motor_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "igbt_temperature": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},

    # --- Continuous signed (keep) ---
    "maxavailableshorttermcharging": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0},
    "motor_torque": {"variable_category": "continuous", "dtype": "Int16", "scale": 1, "round": 0,
                     "note": "Your obs_max=63489 looks like a glitch; clip later"},
}


DTYPE_RULES.update({
    # 1) Cell voltages (pack_cellvoltage_1..576): store mV as UInt16
    # If you are listing explicitly, do each column; if you want, you can also expand in a loop.
    "pack_cellvoltage_1": {
        "variable_category": "continuous",
        "dtype": "UInt16",
        "scale": 1000,     # V -> mV
        "round": 0,
        "note": "Store as mV (V*1000), UInt16"
    },

    # 2) Pack temperatures (pack_temperature1..108): store °C as Int16 (drop decimals)
    "pack_temperature1": {
        "variable_category": "continuous",
        "dtype": "Int16",
        "scale": 1,
        "round": 0,
        "note": "Store as int °C"
    },

    # 3) Pedals: store as UInt8 by dropping decimals
    "brakepedalpos": {
        "variable_category": "continuous",
        "dtype": "UInt8",
        "scale": 1,
        "round": 0,
        "note": "Drop decimals; store 0..100"
    },
    "accelarationpedal": {
        "variable_category": "continuous",
        "dtype": "UInt8",
        "scale": 1,
        "round": 0,
        "note": "Drop decimals; store 0..100"
    },
    "brake_pedal": {
        "variable_category": "continuous",
        "dtype": "UInt8",
        "scale": 1,
        "round": 0,
        "note": "Drop decimals; store 0..102"
    },

    # 4) Counting / life-signal counters: UInt8
    "lifesignalofinsulation": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},
    "dcdclifesignal": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},
    "bmslifesignal": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},
    "lowvoltagedcaclifesignal": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},
    "airpumpdcaclifesignal": {"variable_category": "discrete", "dtype": "UInt8", "scale": 1, "round": 0},

    # 5) Categorical alarms/status enums: UInt8
    "dcdc_statusandfailure": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "batteryhightemperaturealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "battery_function_alarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "batterypackundervoltagealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "batterypackovervoltagealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "monomerovervoltagealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "monomerundervoltagealarm": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
    "hightempalarmofcharginggun": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},

    # 6) insulation_status: you labeled it “binary”; DBC says 0..3.
    # That’s not binary. Treat as categorical UInt8 unless you want a custom mapping.
    "insulation_status": {"variable_category": "categorical", "dtype": "UInt8", "scale": 1, "round": 0},
})

# for i in range(1, 577):
#     DTYPE_RULES.setdefault(f"pack_cellvoltage_{i}", {
#         "variable_category": "continuous",
#         "dtype": "UInt16",
#         "scale": 1000,
#         "round": 0,
#         "note": "Store as mV (V*1000), UInt16"
#     })

# for i in range(1, 109):
#     DTYPE_RULES.setdefault(f"pack_temperature{i}", {
#         "variable_category": "continuous",
#         "dtype": "Int16",
#         "scale": 1,
#         "round": 0,
#         "note": "Store as int °C"
#     })


In [ ]:
def attach_dtype_associations(bounds_df: pd.DataFrame, dtype_rules: dict) -> pd.DataFrame:
    out = bounds_df.copy()

    def lookup(col: str, field: str, default=np.nan):
        spec = dtype_rules.get(col, None)
        if spec is None:
            return default
        return spec.get(field, default)

    out["recommended_dtype"] = out["column"].map(lambda c: lookup(c, "dtype", "UNSPECIFIED"))
    out["variable_category"] = out["column"].map(lambda c: lookup(c, "variable_category", "UNSPECIFIED"))
    out["scale"] = out["column"].map(lambda c: lookup(c, "scale", 1))
    out["round"] = out["column"].map(lambda c: lookup(c, "round", np.nan))
    out["dtype_note"] = out["column"].map(lambda c: lookup(c, "note", ""))

    out["is_mapped"] = out["recommended_dtype"].ne("UNSPECIFIED")
    return out

bounds_df2 = attach_dtype_associations(bounds_df, DTYPE_RULES)

# show unmapped columns for iterative completion
unmapped = bounds_df2.loc[~bounds_df2["is_mapped"], "column"].tolist()
# print("Unmapped columns:", unmapped)

In [ ]:
bounds_df2[bounds_df2.column.isin(unmapped)]

,column,non_null_pct,dbc_min,obs_min,p01,p05,p50,p95,p99,obs_max,dbc_max,recommended_dtype,variable_category,scale,round,dtype_note,is_mapped
97,sequence,100.0,NaN,30.0,958.0,3574.0,32856.0,62529.0,65165.0,65535.0,NaN,UNSPECIFIED,UNSPECIFIED,1,NaN,,False
98,number_of_records,100.0,NaN,1.0,8.0,14.0,34.0,83.0,99.0,137.0,NaN,UNSPECIFIED,UNSPECIFIED,1,NaN,,False
99,number_of_can_ids,100.0,NaN,1.0,3.0,3.0,22.0,26.0,28.0,33.0,NaN,UNSPECIFIED,UNSPECIFIED,1,NaN,,False


In [ ]:
# Create a summary table of dtypes and non-null counts
info_df = pd.DataFrame({
    "Column": df_raw2.columns,
    "Non-Null Count": df_raw2.count().values,
    "Dtype": df_raw2.dtypes.values
})

# Now you can export this new DataFrame to CSV
info_df.to_csv('cpoall_attributes_original.csv', index=False)


In [ ]:
bounds_df2

,column,non_null_pct,dbc_min,obs_min,p01,p05,p50,p95,p99,obs_max,dbc_max,recommended_dtype,variable_category,scale,round,dtype_note,is_mapped
0,pack_cellvoltage_1,0.783270,0.0,3.214,3.23775,3.255,3.315000,3.42500,3.448950,3.46700,600.0,UInt16,continuous,1000,0.0,"Store as mV (V*1000), UInt16",True
1,pack_temperature1,7.400899,-40.0,24.000,25.00000,25.000,27.000000,32.00000,33.000000,33.00000,215.0,Int16,continuous,1,0.0,Store as int °C,True
2,insulation__value,21.924870,0.0,0.000,4210.63000,4998.000,4999.000000,4999.00000,4999.000000,4999.00000,60000.0,UInt16,continuous,1,0.0,Ohms-ish domain; confirm scaling if needed,True
3,lifesignalofinsulation,21.924870,0.0,0.000,2.00000,11.000,127.000000,243.00000,253.000000,255.00000,255.0,UInt8,discrete,1,0.0,,True
4,insulation_status,21.924870,0.0,0.000,0.00000,0.000,0.000000,0.00000,0.000000,0.00000,3.0,UInt8,categorical,1,0.0,,True
5,dcdcbusvoltage,41.472314,0.0,3.000,617.00000,623.000,637.000000,658.00000,663.000000,668.00000,1000.0,UInt16,continuous,1,0.0,,True
6,dcdclifesignal,41.472314,0.0,0.000,2.00000,12.000,126.000000,242.00000,253.000000,255.00000,255.0,UInt8,discrete,1,0.0,,True
7,dcdc_out_put_currant,41.472314,0.0,0.000,7.00000,9.000,20.000000,40.00000,66.000000,118.00000,125.0,Float32,continuous,1,1.0,,True
8,dcdc_out_put_volatge,41.472314,0.0,22.000,27.50000,27.500,27.500000,27.50000,27.500000,27.50000,36.0,Float32,continuous,1,1.0,,True
9,dcdc_statusandfailure,41.472314,0.0,0.000,1.00000,1.000,1.000000,1.00000,1.000000,2.00000,3.0,UInt8,categorical,1,0.0,,True


In [ ]:
bounds_df2.to_csv('cpoall_attributes_coverage.csv')